In [1]:
import pickle
import torch 
import torch.autograd as autograd
import copy



with open('H0H1.pkl', 'rb') as fp:
    problem = pickle.load(fp)
    
import sys
sys.path.append('../')
from models import misc_mnet as mm

Mobj = mm.Meta_Mnet(dim_a=16, batchsize=32, mode='exact')
Mobj.Ms.data = copy.deepcopy(problem['M'])

inner_args= {}
inner_args['num_loops'] = 200
inner_args['detach'] = 1

H0 = copy.deepcopy(problem['H0'])
H1 = copy.deepcopy(problem['H1'])
verbose = 1

In [2]:
print(torch.max(torch.abs(problem['M'])))


Mobj.Ms[0][:4, :4]

tensor(1.0280)


tensor([[ 0.9665, -0.0133, -0.0074,  0.0030],
        [-0.0232,  0.9505, -0.0124, -0.0154],
        [-0.0015,  0.0166,  0.9546, -0.0024],
        [ 0.0035, -0.0080,  0.0109,  0.9575]], grad_fn=<SliceBackward>)

In [3]:
print(torch.sum((H1 - H0 @ Mobj.Ms)**2))
print(Mobj(H0, H1))

tensor(5017.1616, grad_fn=<SumBackward0>)
tensor(70.8319, grad_fn=<SqrtBackward>)


In [5]:
with torch.enable_grad():

    for j in range(inner_args['num_loops']):
        loss = Mobj(H0, H1)
        grads = autograd.grad(loss, Mobj.parameters(),
                              create_graph=(not inner_args['detach']),
                              only_inputs=True, allow_unused=True)
        # parameter update
        for param, grad in zip(Mobj.parameters(), grads):

            #new_param = param - self.inner_args['lr'] * grad
            new_param = param - 0.001 * grad

            param.data.copy_(new_param)
        if verbose: print(f"""inner_loss {loss.item()}""")
    if verbose: print("-"*10)
Mstar = Mobj.Ms.data
print(torch.max(torch.abs(H0)))
print(torch.max(torch.abs(H1)))
print(torch.max(torch.abs(grad)))
print(torch.max(torch.abs(Mstar)))



inner_loss 68.89547729492188
inner_loss 68.7554702758789
inner_loss 68.61869812011719
inner_loss 68.48509216308594
inner_loss 68.35458374023438
inner_loss 68.22713470458984
inner_loss 68.10266876220703
inner_loss 67.98112487792969
inner_loss 67.86245727539062
inner_loss 67.7466049194336
inner_loss 67.63349914550781
inner_loss 67.5230941772461
inner_loss 67.41533660888672
inner_loss 67.31016540527344
inner_loss 67.20751953125
inner_loss 67.10736083984375
inner_loss 67.00962829589844
inner_loss 66.91426086425781
inner_loss 66.82122802734375
inner_loss 66.73046112060547
inner_loss 66.64191436767578
inner_loss 66.5555419921875
inner_loss 66.47129821777344
inner_loss 66.38912200927734
inner_loss 66.3089828491211
inner_loss 66.23081970214844
inner_loss 66.15459442138672
inner_loss 66.08026123046875
inner_loss 66.00778198242188
inner_loss 65.93710327148438
inner_loss 65.86819458007812
inner_loss 65.80101013183594
inner_loss 65.7354965209961
inner_loss 65.671630859375
inner_loss 65.609375
inne

In [6]:
import numpy as np
trueM = np.random.normal(size = (H0.shape[0], 16, 16))
trueH1 = H0 @ trueM
inner_args['num_loops']=100

In [7]:
print(torch.sum((trueH1 - H0)**2))


Mobj = mm.Meta_Mnet(dim_a=16, batchsize=32, mode='exact')
Mobj.Ms.data = copy.deepcopy(problem['M'])
with torch.enable_grad():

    for j in range(inner_args['num_loops']):
        loss = Mobj(H0, trueH1)
        grads = autograd.grad(loss, Mobj.parameters(),
                              create_graph=(not inner_args['detach']),
                              only_inputs=True, allow_unused=True)
        # parameter update
        for param, grad in zip(Mobj.parameters(), grads):

            #new_param = param - self.inner_args['lr'] * grad
            new_param = param - 0.1 * grad

            param.data.copy_(new_param)
        if verbose: print(f"""inner_loss {loss.item()}""")
    if verbose: print("-"*10)
Mstar = Mobj.Ms.data

tensor(8031373.0520, dtype=torch.float64)
inner_loss 2832.312918718051
inner_loss 2737.3636272374642
inner_loss 2642.763533073331
inner_loss 2548.5250797533727
inner_loss 2454.661476738047
inner_loss 2361.186762803141
inner_loss 2268.1158796579502
inner_loss 2175.4647600843173
inner_loss 2083.2504214832456
inner_loss 1991.4910725242457
inner_loss 1900.2062380505674
inner_loss 1809.416896398795
inner_loss 1719.1456378249768
inner_loss 1629.4168492704664
inner_loss 1540.2569265213988
inner_loss 1451.694514149948
inner_loss 1363.7608021914873
inner_loss 1276.4898585430024
inner_loss 1189.91903328062
inner_loss 1104.0894457175048
inner_loss 1019.0465719021612
inner_loss 934.8409574371425
inner_loss 851.5291345096713
inner_loss 769.1747515051326
inner_loss 687.8500623646406
inner_loss 607.6379070176282
inner_loss 528.634430136876
inner_loss 450.95303723087466
inner_loss 374.73038794432
inner_loss 300.1363551309704
inner_loss 227.39247164796134
inner_loss 156.81274314071018
inner_loss 88.928

In [8]:
trueM[0][:4, :4]

array([[-0.99521988, -1.56781337,  0.44956645, -0.7920913 ],
       [ 0.55087646,  0.37448973, -0.70049339,  0.99912825],
       [ 0.63325988,  0.73153545, -1.321742  ,  0.11886802],
       [ 0.99759704,  1.34992008,  0.63435115, -0.5054478 ]])

In [9]:
Mobj.Ms[0][:4, :4]

tensor([[-0.9951, -1.5679,  0.4495, -0.7921],
        [ 0.5509,  0.3745, -0.7005,  0.9991],
        [ 0.6332,  0.7316, -1.3217,  0.1189],
        [ 0.9978,  1.3498,  0.6343, -0.5055]], grad_fn=<SliceBackward>)